# Advent of Code 2022 - Day 12

## Part One

Some form of path finding algorithm is required I think, such as:

- (https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2)[https://medium.com/@nicholas.w.swift/easy-a-star-pathfinding-7e6689c7f7b2]
- (https://gist.github.com/ryancollingwood/32446307e976a11a1185a5394d6657bc)[https://gist.github.com/ryancollingwood/32446307e976a11a1185a5394d6657bc]


In [1]:
from pathlib import Path
import numpy as np
import heapq
from typing import List, Dict, Set, Tuple, Any
from warnings import warn

In [2]:
INPUT_FILE = "day12_input_example.txt"


Read the input in to a matrix

In [3]:
def read_input(input_name: str) -> List[str]:
    input_path = Path.cwd() / "input" / input_name

    with open(input_path, "r") as input_file:

        lines = input_file.readlines()

    return [line.rstrip("\n") for line in lines]


def str_to_str_list(line: str) -> List[int]:

    return [char for char in line]


def input_to_matrix(line_list: List[str]) -> List[List[int]]:

    list_matrix = [str_to_str_list(line) for line in line_list]

    return np.matrix(list_matrix)

In [4]:
height_matrix = input_to_matrix(read_input(INPUT_FILE))
height_matrix


matrix([['S', 'a', 'b', 'q', 'p', 'o', 'n', 'm'],
        ['a', 'b', 'c', 'r', 'y', 'x', 'x', 'l'],
        ['a', 'c', 'c', 's', 'z', 'E', 'x', 'k'],
        ['a', 'c', 'c', 't', 'u', 'v', 'w', 'j'],
        ['a', 'b', 'd', 'e', 'f', 'g', 'h', 'i']], dtype='<U1')

Create a Node Class

In [5]:
class Node:
    def __init__(self, letter: str, parent=None, position=None) -> None:
        self.letter = letter
        self.parent = parent
        self.position = position

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        return self.position == other.position

    def __repr__(self):
        return f"{self.position} - g: {self.g} h: {self.h} f: {self.f}"

    # defining less than for purposes of heap queue
    def __lt__(self, other):
        return self.f < other.f

    # defining greater than for purposes of heap queue
    def __gt__(self, other):
        return self.f > other.f

Create the A* algorithm

In [6]:
def return_path(current_node):
    path = []
    current = current_node
    while current is not None:
        path.append(current.position)
        current = current.parent
    return path[::-1]   # Return a reversed path

In [7]:
def astar(
    maze: np.matrix,
    start: Tuple[int, int],
    end: Tuple[int, int],
) -> List[Tuple[int, int]]:

    # Create start and end nodes
    start_node = Node("S", None, start)
    start_node.g = start_node.h = start_node.f = 0
    end_node = Node("E", None, end)
    end_node.g = end_node.h = end_node.f = 0

    # Initialise the open and closed lists
    open_list = []
    closed_list = []

    # Heapify the open_list and Add the start node
    heapq.heapify(open_list)
    heapq.heappush(open_list, start_node)

    # Add a stop condition
    outer_iterations = 0
    max_iterations = (maze.shape[0] * maze.shape[1]) // 2

    # What squares do we search
    adjacent_squares = (
        (0, -1),
        (0, 1),
        (-1, 0),
        (1, 0),
    )

    # Loop until we find the end
    while len(open_list) > 0:
        outer_iterations += 1

        current_node: Node

        if outer_iterations > max_iterations:
            # if we hit this point return the path such as it is
            # it will not contain the destination
            warn("giving up on pathfinding too many iterations")
            return return_path(current_node)

        # Get the current node
        current_node = heapq.heappop(
            open_list
        )  # Pops the smallest item == highest priority
        closed_list.append(current_node)

        # Found the goal!
        if current_node == end_node:
            print("success!")
            return return_path(current_node)

        # Generate children
        children = []

        for (
            new_position
        ) in adjacent_squares:  # Cycle through adjacent squares to current

            # Get Node Position (offset current by the adjacent square relative position)
            node_position = (
                current_node.position[0] + new_position[0],
                current_node.position[1] + new_position[1],
            )

            # Make sure it's in range
            if (
                node_position[0] > (maze.shape[0] - 1)
                or node_position[0] < 0
                or node_position[1] > (maze.shape[1] - 1)
                or node_position[1] < 0
            ):
                print(f"{node_position}")
                
                print("Not in range")
                continue

            # Make sure terrain is climable (letter is at most 1 higher than current, lower is ok)
            node_letter = maze[node_position[0],node_position[1]]
            if (
                node_position != start_node.position
                and ord(node_letter) >= ord(current_node.letter) + 2
            ):
                # print("Can't climb")
                continue

            # Create a new node
            new_node = Node(node_letter, current_node, node_position)

            # Append
            children.append(
                new_node
            )  # This is a list of all possible positions from current

        child: Node
        for child in children:
            # Check if Child is on the closed list
            if (
                len(
                    [
                        closed_child
                        for closed_child in closed_list
                        if closed_child == child
                    ]
                )
                > 0
            ):
                continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + (
                (child.position[1] - end_node.position[0]) ** 1
            )  # a^2 = b^2 + c^2
            child.f = child.g + child.h
            
            # Check if child is on the open list
            if len([open_node for open_node in open_list if child.position == open_node.position and child.g > open_node.g]) > 0:
                continue

            # Add the child to the open list
            heapq.heappush(open_list, child)
            
    warn("Couldn't get a path to destination")
    return None

Let's try it...

In [8]:
maze = height_matrix
start = (0,0)

def find_end(maze: np.matrix) -> str:
    
    for row in range(0,height_matrix.shape[0]):
        for col in range(0, height_matrix.shape[1]):
            if height_matrix[row,col] == "E":
                return (row, col)
            
end = find_end(height_matrix)

In [9]:
path = astar(maze, start, end)

(0, -1)
Not in range
(-1, 0)
Not in range


C:\Users\mb28\AppData\Local\Temp\ipykernel_25908\3048753069.py:127: UserWarning: Couldn't get a path to destination
  warn("Couldn't get a path to destination")
